In [1]:
import pymongo
from pymongo import MongoClient
import os, io, json, glob, time
import pprint

In [2]:
# Define database globally
client = MongoClient('mongodb://localhost:27017/')
db = client.election_tweets
tweet_collection = db.tweets
user_collection = db.users

In [3]:
def test():
    print tweet_collection
    print db
test()

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'election_tweets'), u'tweets')
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'election_tweets')


## Minimize Tweets

There is a lot of unnessecary information. The following function remove these keys from the dictionary and also seperate the retweet and user object from the original tweet

In [4]:
#class TweetDB:
#     def __init__(self):
#         client = MongoClient('mongodb://localhost:27017/')
#         db = client.election_tweets
#         self.db = db
#         self. = db.tweets
#         self. = db.users
    
def convert_datetime(twitter_time):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(twitter_time,'%a %b %d %H:%M:%S +0000 %Y'))

def minimize_tweet(tweet):
    # if a tweet doesn't have an idea return none
    if tweet.get('id', None) is None or tweet is None:
        return None
    
    # helper func to remove unnessecary keys
    
    tweet_keys = ['contributors', 'geo', 'lang', 'id_str', 'metadata', 'in_reply_to_status_id_str',
                'in_reply_to_user_id_str', 'quoted_status_id_str', 'notifications', 'truncated',
                  'translator_type', 'contributors_enabled', 'default_profile', 'geo_enabled', 
                'has_extended_profile', 'source', 'coordinates', 'filter_level', 'possibly_sensitive', 
                'quoted_status_id_str','scopes', 'withheld_copyright']
    
    user_keys = [ 'contributors_enabled', 'default_profile', 'default_profile_image', 'follow_request_sent',
                'geo_enabled', 'id_str', 'is_translator', 'lang', 'listed_count', 'notifications',
                'profile_background_color', 'profile_banner_url', 'profile_background_tile',
                'profile_background_image_url_https', 'profile_background_image_url', 'profile_image_url',
                'profile_image_url_https', 'profile_image_url_https', 'profile_link_color', 
                'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color',
                'profile_use_background_image', 'show_all_inline_media', 'protected', 'url', 'utc_offset', 
                'time_zone', 'withheld_in_countries', 'withheld_scope']
    
    place_keys = ['attributes', 'bounding_box', 'contained_within', 'place_type', 'url']
    
    def remove_keys(obj, keys):
        if obj is None:
            return None
        
        for key in keys:
            obj.pop(key, None)
        
        # also sets id to _id for mongo
        obj['_id'] = obj.pop('id')
        
        
        
        return obj
    
    # remove keys
    tweet = remove_keys(tweet, tweet_keys)
    
    # handle entities
    entities = tweet.get('entities', None)
    if entities:
        # remove entities from tweet
        tweet.pop('entities', None)
        # remove urls from entities
        entities.pop('urls', None)
        
        # set entity objects directly to tweet
        for entity, items in entities.iteritems():
            # reduce keys from user mentions
            if entity == 'user_mentions':
                for i, obj in enumerate(items):
                    items[i].pop('id_str')
                    items[i].pop('name')
                    items[i]['_id'] = items[i].pop('id')
                    
            if len(items) == 0:
                tweet[entity] = None
            else:
                tweet[entity] = items
                
    # minimize the place field
    tweet['place'] = remove_keys(tweet.get('place', None), place_keys)
    
    # minimize user field
    user = remove_keys(tweet.get('user', None), user_keys)
    if user:
        # set user field in tweet to user id
        tweet['user'] = user.get('_id', None)
    
    ### RETWEET has yet to be
    # check if the tweet was retweeted
    retweet = tweet.get('retweeted_status', None)
    
    # Set to None by default
    retweet_container = None
    
    if retweet:
        # grab the retweet id
        retweet_id = retweet.get('id', None)
        
        # Create retweet container to return
        retweet_container = {
            'retweet': retweet,
            'id': retweet_id,
            # save the time the retweet was made, can use to compare vs a tweet saved in the data base
            'created_at': convert_datetime(tweet.get('created_at'))
        }
        
        # set retweeted_status to the id of the orginal retweet
        tweet['retweeted_status'] = retweet_id
    
    return (tweet, user, retweet_container)

Test JSON file, used for testing minimizing the tweets

In [5]:
test_file_path = './queries/#DonaldTrump OR #HillaryClinton OR Trump OR Clinton OR #Trump OR #Clinton/Canada/2016-11-07T19:43:48.144881.json'
test = io.open(test_file_path)
tweets = json.loads(test.read())
# pprint.pprint([tweet for key, tweet in tweets.iteritems()][:2])
min_tweets = [minimize_tweet(tweet) for key, tweet in tweets.iteritems()]
pprint.pprint(min_tweets[:2])

[({'_id': 795670816545456129,
   u'created_at': u'Mon Nov 07 16:54:40 +0000 2016',
   u'favorite_count': 0,
   u'favorited': False,
   u'hashtags': None,
   u'in_reply_to_screen_name': None,
   u'in_reply_to_status_id': None,
   u'in_reply_to_user_id': None,
   u'is_quote_status': False,
   u'place': {'_id': u'0064529d5fb32582',
              u'country': u'Canada',
              u'country_code': u'CA',
              u'full_name': u'Georgian Bluffs, Ontario',
              u'name': u'Georgian Bluffs'},
   u'retweet_count': 1,
   u'retweeted': False,
   u'symbols': None,
   u'text': u'Trump interrupts summation by re-looping back into the full, rambling glory of the body of his speech.',
   u'user': 167535120,
   u'user_mentions': None},
  {'_id': 167535120,
   u'created_at': u'Fri Jul 16 21:00:58 +0000 2010',
   u'description': u'I write columns for the National Post and Postmedia/Sun Media newspapers.',
   u'entities': {u'description': {u'urls': []}},
   u'favourites_count': 10029,
   

In [6]:
# Load place id look up table

# open file
f = io.open('places_id.json')
# load the json saved in the file
places = json.loads(f.read())

# Create the places look up table
places_lookup = {}
for place in places:
    place_id = place['place_id']
    country = place['twitter_name']
    places_lookup[place_id] = country
    
print places_lookup

{u'dd9c0d7d7e07eb49': u'South Africa', u'25530ba03b7d90c6': u'Mexico', u'3376992a082d67c7': u'Canada', u'12ff487cb5a05ff7': u'Central African Republic', u'98c7a0e48eb6b42b': u'South Georgia and the South Sandwich Islands', u'fc6282dff859b848': u'\u0627\u0644\u0645\u0645\u0644\u0643\u0629 \u0627\u0644\u0623\u0631\u062f\u0646\u064a\u0629 \u0627\u0644\u0647\u0627\u0634\u0645\u064a\u0629', u'3f14ce28dc7c4566': u'Australia', u'333a5811d6b0c1cb': u'Hviderusland', u'c027ef7e8f9b3ee2': u'French Southern and Antarctic Lands', u'f3bfc7dcc928977f': u'Frankrig', u'c29833e68a86e703': u'Danmark', u'991b4344edc2d520': u'Fransk Guyana', u'08b2d74428e2ca88': u'Saudi Arabien', u'88f6c97979c5c22d': u'British Indian Ocean Territory', u'ea5eca163987d547': u'French Polynesia', u'5714382051c06d1e': u'\u0420\u043e\u0441\u0441\u0438\u044f', u'08fbcf4cbc4bc5b9': u'United States Minor Outlying Islands', u'82b141af443cb1b8': u'Sverige', u'ac26bceca6c10474': u'Cuba', u'b850c1bfd38f30e0': u'Indien', u'3f63906fc8aa5

Iterate through queries before election

In [7]:
def insert_many(filename, query):
    
    def insert(document, query, collection):
        
        _id = document['_id']
        # find a cursor with the same id
        cur = collection.find({'_id': _id}).limit(1)
        
        #
        # if finding the most updated tweets is not required
        # can pass upsert=True to collection.update
        #
        
        # make a new entry
        if document and cur.count() == 0:
            # save query to database in list
            document['query'] = [query]
            collection.insert_one(document)
        # check if entry should be updated
        else:
            new_document = cur.next()
            # convert time objects to date time to compare
            old_time = convert_datetime(document['created_at'])
            new_time = convert_datetime(new_document['created_at'])
            
            # if the new tweet is more recent, then update
            if new_time > old_time:
                # in case the new queries are different
                old_query = document['query']
                new_query = new_document['query']
                if new_query not in old_query:
                    new_document['query'].append(new_query)
                collection.update({{'_id': _id}}, new_document)

        return None

    f = io.open(filename)
    tweets = json.loads(f.read())
    for tweet_id, raw_tweet in tweets.iteritems():
        tweet, user, retweet_container = minimize_tweet(raw_tweet)
        # insert tweet and user
        insert(tweet, query, tweet_collection)
        insert(user, query, user_collection)
        
        # if there is a retweet
        if retweet_container:
            tweet, user, _ = minimize_tweet(retweet_container['retweet'])
            # insert retweet and user
            insert(tweet, query, tweet_collection)
            insert(user, query, user_collection)
            
    f.close()
    return None

In [8]:
def create_tweet_db(root, path):
    '''
    Retrieve all the tweets from the comments, minimize and create database.
    '''
    os.chdir(path)
    root_query = os.getcwd()

    for query in os.listdir(os.getcwd()):
        # skip hidden files
        if query.startswith('.') or query.endswith('.json'):
            continue

        # move into query directory
        os.chdir(query)
        # save the path of the current query
        query_path = os.getcwd()

        # retrieve all the json files not in location directories
        for filename in glob.glob('*.json'):
            insert_many(filename, query)
            # print query, json_file

        # iterate through all subdirectories inside of the parent query
        for subdir in os.listdir(os.getcwd()):
            # skip hidden files and json files (since we already retrieved these above)
            if subdir.startswith('.') or subdir.endswith('.json'):
                continue
            # move into sub directory inside of query (country, timestamp)
            os.chdir(subdir)

            # retrieve all the json files not in location and timestamp folders
            for filename in glob.glob('*.json'):
                insert_many(filename, query)
                # print query, subdir, json_file

            # move out back to query directory
            os.chdir(query_path)

        # move out of query directory
        os.chdir(root_query)

    os.chdir(root)

In [9]:
# set the current directory
path = 'queries'
root = os.getcwd()

create_tweet_db(root, path)

In [10]:
os.chdir(root)

In [11]:
os.getcwd()

'/Users/brydenfogelman/DTU/social-graphs/02805-social-graphs/project'

In [17]:
tweet_collection.count()

112948

In [18]:
user_collection.count()

87584

In [14]:
path = 'election_day'
create_tweet_db(root, path)

In [16]:
tweet_collection.stats()

TypeError: 'Collection' object is not callable. If you meant to call the 'stats' method on a 'Collection' object it is failing because no such method exists.